In [ ]:
from watchlist_data import DataRetriever
from yfdownloader import get_ohlc
import numpy as np
import pandas as pd
import plotly.io as pio
pio.renderers.default = 'notebook'
import plotly.graph_objs as go
from plotly.subplots import make_subplots


def _get_df(file,itvl='1d'):
    with open(file,'r') as f:
        stocks = [i.strip() for i in f.readlines()]
    
    data = []
    for stock in stocks:
        try:
            data.append(get_ohlc(stock,itvl)['Close'])
        except Exception as e:
            continue
    return pd.concat(data,axis=1,join='inner')

def run(file,itvl='1d',sdate='2020-05-01'):
    ma_len = {
        '1d' : 252,
        '1wk' : 52,
        '1mo' : 12
    }
    sp = get_ohlc('^GSPC',itvl)['Close']
    dax = get_ohlc('^GDAXI',itvl)['Close']
    cac = get_ohlc('^FCHI',itvl)['Close']
    ftse = get_ohlc('^FTSE',itvl)['Close']
    indices = pd.concat([sp,dax,cac,ftse],axis=1,join='inner')
    indices = indices.loc[indices.index>=sdate]
    
    df = _get_df(file,itvl)
    df = df.loc[df.index>=sdate]
    
    indices_logr = np.log(indices / indices.shift(1)).fillna(0.0).mean(axis=1)
    indices_cum_returns = np.exp(indices_logr.cumsum()) - 1
    
    logr = np.log(df / df.shift(1)).fillna(0.0).mean(axis=1)
    cum_returns = np.exp(logr.cumsum()) - 1
    MA_200 = cum_returns.rolling(ma_len[itvl]).mean().dropna()
    VaR = logr + 1.65 * logr.std() * 100
    
    indices_r_year = pd.Series(dtype='float')
    r_year = pd.Series(dtype='float')

    for year, group in indices_logr.groupby(indices_logr.index.year):
        indices_r_year.loc[year] = round(np.exp(group.cumsum()).iloc[-1] - 1, 4)

    for year, group in logr.groupby(logr.index.year):
        r_year.loc[year] = round(np.exp(group.cumsum()).iloc[-1] - 1, 4)

    COLORS = {
        'primary': '#6366f1',
        'secondary': '#22d3ee',
        'accent': '#a78bfa',
        'positive': '#10b981',
        'negative': '#ef4444',
    }
    fig = make_subplots(
        rows=2, 
        cols=1,
        row_heights=[0.68, 0.32],
        vertical_spacing=0.1
    )
    fig.add_trace(
        go.Scatter(
            x=cum_returns.index,
            y=cum_returns * 100,
            mode='lines',
            name='Portfolio',
            line=dict(color=COLORS['primary'], width=2.5, shape='spline'),
            fill='tozeroy',
            fillcolor='rgba(99, 102, 241, 0.15)',
            hovertemplate='%{x|%Y-%m-%d}<br>%{y:.2f}%<extra></extra>',
        ), 
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=indices_cum_returns.index,
            y=indices_cum_returns * 100,
            mode='lines',
            name='Benchmark',
            line=dict(color=COLORS['secondary'], width=2, shape='spline'),
            hovertemplate='%{x|%Y-%m-%d}<br>%{y:.2f}%<extra></extra>',
        ), 
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=MA_200.index,
            y=MA_200 * 100,
            mode='lines',
            name=f'MA ({ma_len[itvl]})',
            line=dict(color=COLORS['accent'], width=1.5, shape='spline', dash='dash'),
            hovertemplate='%{x|%Y-%m-%d}<br>%{y:.2f}%<extra></extra>',
        ), 
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=VaR.index,
            y=VaR,
            mode='lines',
            name='VaR',
            line=dict(color='#f59e0b', width=2, shape='spline'),
            hovertemplate='%{x|%Y-%m-%d}<br>%{y:.2f}%<extra></extra>',
        ), 
        row=2, col=1
    )
    fig.update_layout(
        template='plotly_dark',
        title=dict(text="Cumulative Portfolio Return", x=0.5, font=dict(size=20)),
        font=dict(size=12, family='Arial, sans-serif'),
        height=650,
        margin=dict(l=60, r=40, t=80, b=60),
        hovermode='x unified',
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5)
    )
    fig.update_xaxes(showgrid=True, gridcolor='#1e293b', row=1, col=1)
    fig.update_xaxes(title='Date', showgrid=True, gridcolor='#1e293b', row=2, col=1)
    fig.update_yaxes(title='Return (%)', showgrid=True, gridcolor='#1e293b', zeroline=True, row=1, col=1)
    fig.update_yaxes(title='VaR (%)', showgrid=True, gridcolor='#1e293b', zeroline=True, row=2, col=1)
    fig.show()
    
    fig2 = make_subplots(
        rows=2, cols=1,
        row_heights=[0.5, 0.5],
        vertical_spacing=0.12,
        subplot_titles=('Portfolio', 'Benchmark')
    )
    fig2.add_trace(
        go.Bar(
            x=r_year.index.astype(str),
            y=r_year * 100,
            marker=dict(
                color=[COLORS['positive'] if ret >= 0 else COLORS['negative'] for ret in r_year],
                line=dict(color='#fff', width=0.5)
            ),
            hovertemplate='%{x}<br>%{y:.1f}%<extra></extra>',
            showlegend=False
        ), 
        row=1, col=1
    )
    fig2.add_trace(
        go.Bar(
            x=indices_r_year.index.astype(str),
            y=indices_r_year * 100,
            marker=dict(
                color=[COLORS['positive'] if ret >= 0 else COLORS['negative'] for ret in indices_r_year],
                line=dict(color='#fff', width=0.5)
            ),
            hovertemplate='%{x}<br>%{y:.1f}%<extra></extra>',
            showlegend=False
        ), 
        row=2, col=1
    )
    fig2.update_layout(
        template='plotly_dark',
        title=dict(text="Annual Returns", x=0.5, font=dict(size=20)),
        font=dict(size=12, family='Arial, sans-serif'),
        height=650,
        margin=dict(l=60, r=40, t=80, b=60),
        bargap=0.2
    )
    fig2.update_xaxes(showgrid=False, row=1, col=1)
    fig2.update_xaxes(title='Year', showgrid=False, row=2, col=1)
    fig2.update_yaxes(title='Return (%)', showgrid=True, gridcolor='#1e293b', zeroline=True, row=1, col=1)
    fig2.update_yaxes(title='Return (%)', showgrid=True, gridcolor='#1e293b', zeroline=True, row=2, col=1)
    fig2.show()
    
    
file = 'stocklists/filtered.txt'
run(file,'1mo')